In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import special

## Small example for binomial coefficient nCr using scipy

In [2]:
B=special.binom(6, 3) #binom(n,r)
print(B)

20.0


## Aders

In [3]:
def FindSpan(n_inp,degree_inp,u_inp,knot_vector_inp):
    x=knot_vector_inp[degree_inp+1]
    if (u_inp < x):
        return degree_inp
    else:
        for i,pos in enumerate(knot_vector_inp):
            if math.floor(u_inp) == pos:
                return (i)

In [4]:
# Derivatives are stored in ders[k][j]
# ders[k][j] is the kth derivative of function N_(i-p+j,p)
# If k>p the derivatives are zero.
def DersBasisFuns(i,u,p,m,U):
    #Inititation of dimentions for 2D matrices
    ndu=np.zeros((p+1,p+1))
    ders=np.zeros((p+1,p+1))
    a=np.zeros((2,p+1))
    left =np.zeros(p+2)
    right =np.zeros(p+2)
    
    ndu[0][0]=1.0
    for j in range(1,p+1):
        left[j] = u - U[i+1-j]
        right[j] = U[i+j] - u
        saved=0.0
        for r in range(j):
            #Lower triangle
            ndu[j][r] = right[r+1]+left[j-r]
            temp=ndu[r][j-1]/ndu[j][r]
            #Upper triangle
            ndu[r][j] = saved+(right[r+1]*temp)
            saved=left[j-r]*temp
        ndu[j][j] = saved
    for j in range (p+1): #Load the basis functions
        ders[0][j]=ndu[j][p]
    #This secion computes the derivatives
    for r in range(p+1):
        s1=0
        s2=1 #Alternative rows in array a
        a[0][0] = 1.0
        #Loop to compute kth derivative
        for k in range(1,m+1):
            d=0.0
            rk=r-k
            pk=p-k
            if(r>=k):
                a[s2][0]=a[s1][0]/ndu[pk+1][rk]
                d=a[s2][0]*ndu[rk][pk]
            if(rk>=-1):
                j1=1
            else:
                j1=-rk
            if(r-1<=pk):
                j2=k-1
            else:
                j2=p-r
            for j in range (j1,j2+1):
                a[s2][j] =(a[s1][j]-a[s1][j-1])/ndu[pk+1][rk+j]
                d += (a[s2][j]*ndu[rk+j][pk])
            if(r<=pk):
                a[s2][k]=-a[s1][k-1]/ndu[pk+1][r]
                d+=(a[s2][k]*ndu[r][pk])
            ders[k][r]=d
            #Switch rows
            j=s1
            s1=s2
            s2=j
            #Multiply through by the correct factors
    r=p
    for k in range(1,m+1):
        for j in range(p+1):
            ders[k][j] =ders[k][j]* r
        r =r* (p-k)
    return ders

In [5]:
#An extra W is given as input to function for convinience which contain weights of corresponding control points
def SurfaceDerivsAlgAuv(n,p,U,m,q,V,P,W,u,v,d):
    SKL=np.zeros((p+1,q+1,3))
    temp=np.zeros((q+1,3))
    #print(SKL)
    #print(temp[0])
    #print(P[1][1])
    du = min(d,p)
    for k in range(p+1,d+1):
        for l in range(d-k+1):
            SKL[k][l]=0.0
    dv = min(d,p)
    for l in range(q+1,d+1):
        for k in range(d-l+1):
            SKL[k][l]=0.0 
    uspan=FindSpan(n,p,u,U)
    unders=DersBasisFuns(uspan,u,p,du,U)
    vspan=FindSpan(m,q,v,V)
    vnders=DersBasisFuns(vspan,v,q,dv,V)
    for k in range(du+1):
        for s in range(q+1):
            temp[s] = 0.0
            for r in range(p+1):
                temp[s]=temp[s] + unders[k][r]*P[uspan-p+r][vspan-q+s]*W[uspan-p+r][vspan-q+s]
            dd=min(d-k,dv)
            for l in range(dd+1):
                SKL[k][l] = 0.0
                for s in range(q+1):
                    SKL[k][l] = SKL[k][l] + vnders[l][s]*temp[s]
    return SKL #Returning Aders here

## wders

In [6]:
def SurfaceDerivsAlgWuv(n,p,U,m,q,V,P,W,u,v,d):
    SKL=np.zeros((p+1,q+1,1)) #Have to cross check if this is right way of dimensioning SKL
    temp=np.zeros((q+1,1))    # Have to be because W is a scalar unlike P which is vector
    #print(SKL)
    #print(temp[0])
    #print(P[1][1])
    du = min(d,p)
    for k in range(p+1,d+1):
        for l in range(d-k+1):
            SKL[k][l]=0.0
    dv = min(d,p)
    for l in range(q+1,d+1):
        for k in range(d-l+1):
            SKL[k][l]=0.0 
    uspan=FindSpan(n,p,u,U)
    unders=DersBasisFuns(uspan,u,p,du,U)
    vspan=FindSpan(m,q,v,V)
    vnders=DersBasisFuns(vspan,v,q,dv,V)
    for k in range(du+1):
        for s in range(q+1):
            temp[s] = 0.0
            for r in range(p+1):
                temp[s]=temp[s] + unders[k][r]*W[uspan-p+r][vspan-q+s]
            dd=min(d-k,dv)
            for l in range(dd+1):
                SKL[k][l] = 0.0
                for s in range(q+1):
                    SKL[k][l] = SKL[k][l] + vnders[l][s]*temp[s]
    return SKL #Returning wders here

## Function to extract derivatives of NURBS Surface

### Parameters

In [7]:
U = np.array([0., 0., 0., 1., 2., 3., 4., 4., 5., 5., 5.])
V = np.array([0., 0., 0., 1., 2., 3., 3., 3.])
u=2.5
v=1
p=2
q=2
n=(np.size(U)-1)-p-1
m=(np.size(V)-1)-q-1
d=1 # 1st deivative in both directions , d greater than p and q is allowed
##The below control points are random but important part is adapted from NURBS book 133 page
#               [[0.,2.,4.,1.],[0.,6.,4.,2.],[0.,2.,0.,1.]]
#               [[4.,6.,8.,2.],[12.,24.,12.,6.],[4.,6.,0.,2.]]
#               [[4.,2.,4.,1.],[8.,6.,4.,2.],[4.,2.,0.,1.]]
P_W=np.array([[[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,6.,4.,2.],[0.,2.,0.,1.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[4.,6.,8.,2.],[12.,24.,12.,6.],[4.,6.,0.,2.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[4.,2.,4.,1.],[8.,6.,4.,2.],[4.,2.,0.,1.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.]]])
# Control point vector (P_W_1) with all weights equal to 1
P_W_1=np.array([[[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,6.,4.,1.],[0.,2.,0.,1.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[4.,6.,8.,1.],[12.,24.,12.,1.],[4.,6.,0.,1.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[4.,2.,4.,1.],[8.,6.,4.,1.],[4.,2.,0.,1.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.]],
            [[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.],[0.,2.,4.,1.]]])
P=P_W[:,:,0:3]
W=P_W[:,:,3]
P_1=P_W_1[:,:,0:3]
W_1=P_W_1[:,:,3]

In [8]:
def RatSurfaceDerivs(Aders,wders,d):
    SKL=np.zeros((p+1,q+1,3))
    for k in range(d+1):
        for l in range(d-k+1):
            v=Aders[k][l]
            for j in range(1,l+1):
                v=v-special.binom(l,j)*wders[0][j]*SKL[k][l-j]
            for i in range(1,k+1):
                v=v-special.binom(k,i)*wders[i][0]*SKL[k-i][l]
                v2 = 0.0
                for j in range(1,l+1):
                    v2=v2+special.binom(l,j)*wders[i][j]*SKL[k-i][l-j]
                v = v-special.binom(k,i)*v2
            SKL[k][l] = v/wders[0][0]
    return SKL

## Test 1 with different weights

In [9]:
Aders = SurfaceDerivsAlgAuv(n,p,U,m,q,V,P,W,u,v,d)
wders = SurfaceDerivsAlgWuv(n,p,U,m,q,V,P,W,u,v,d)

In [10]:
NURBS_Surface_Derivative = RatSurfaceDerivs(Aders,wders,d)
print(NURBS_Surface_Derivative[0][0])

[ 9.25925926 17.85185185 10.22222222]


## Test 2 with equal weights with value 1
### Expected result : Results same as function SurfaceDerivsAlg1

In [11]:
Aders = SurfaceDerivsAlgAuv(n,p,U,m,q,V,P_1,W_1,u,v,d)
wders = SurfaceDerivsAlgWuv(n,p,U,m,q,V,P_1,W_1,u,v,d)

In [12]:
NURBS_Surface_Derivative = RatSurfaceDerivs(Aders,wders,d)
print(NURBS_Surface_Derivative[0][0])

[ 6.75 12.25  8.5 ]


## $***IMP***$ Check the result with any available code online $***IMP***$